<a href="https://colab.research.google.com/github/ducleins/eurusd_2026/blob/main/02_Notebook/02_Clean_Sheet1_Dealer_CTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================================================
# CELL 1: SETUP VÀ KHAI BÁO BIẾN
# =========================================================

import pandas as pd
import numpy as np
import os
from google.colab import drive

# 1. Kết nối Google Drive (Chỉ chạy nếu Colab chưa kết nối)
# drive.mount('/content/drive')

# 2. Định nghĩa các đường dẫn chính
BASE_PATH = '/content/drive/MyDrive/EURUSD_2026'
RAW_CFTC_PATH = os.path.join(BASE_PATH, '01_Raw', 'cftc')
FILTERED_PATH = os.path.join(BASE_PATH, '01_Raw', 'filtered')

# Đảm bảo thư mục đầu ra tồn tại
os.makedirs(FILTERED_PATH, exist_ok=True)

# 3. Ánh xạ tên cột gốc sang tên chuẩn (Dựa trên 4 file thành công)
COLUMN_MAPPING = {
    'Report_Date_as_YYYY-MM-DD': 'Date',
    'Open_Interest_All': 'Open_Interest_All',
    'Dealer_Positions_Long_All': 'Dealer_Long_All',
    'Dealer_Positions_Short_All': 'Dealer_Short_All',
    'Lev_Money_Positions_Long_All': 'CTA_Long_All',
    'Lev_Money_Positions_Short_All': 'CTA_Short_All',
    'Market_and_Exchange_Names': 'Market_and_Exchange_Names' # Cột quan trọng để lọc
}

In [2]:
# =========================================================
# CELL 2: HÀM XỬ LÝ DỮ LIỆU VÀ TÍNH TOÁN CHỈ SỐ CƠ BẢN
# =========================================================

def process_cftc_file(file_path):
    """
    Chức năng: Đọc file CFTC, lọc theo 'EURO FX', tính Dealer/CTA Net/Pct.
    Xử lý linh hoạt cho file có/không có header.
    """

    # 1. THỬ ĐỌC FILE (Thử với header trước, delimiter là ',')
    try:
        df = pd.read_csv(file_path, sep=',', encoding='utf-8')
        # Kiểm tra sự tồn tại của cột lọc. Nếu không có, có thể header bị lỗi.
        if COLUMN_MAPPING['Market_and_Exchange_Names'] not in df.columns:
            raise ValueError("Lỗi tiêu đề.")
        is_header_present = True

    except (KeyError, ValueError, pd.errors.ParserError):
        # 1b. THỬ ĐỌC LẠI MÀ KHÔNG CÓ HEADER
        df = pd.read_csv(file_path, sep=',', encoding='utf-8', header=None)
        is_header_present = False

        # Gán tên cột theo Index cho các cột cần thiết (Dựa trên vị trí đã kiểm tra)
        COL_INDICES_FOR_FILTER = {
            0: 'Market_and_Exchange_Names', # Cột 0: Tên Hàng Hóa/Tiền Tệ
        }
        df.rename(columns=COL_INDICES_FOR_FILTER, inplace=True)

    # 2. LỌC DỮ LIỆU CỦA EURUSD
    # Lọc theo mã chuẩn 'EURO FX'
    df_euro = df[df['Market_and_Exchange_Names'].str.contains('EURO FX', na=False, case=False)].copy()

    if df_euro.empty:
        raise ValueError("Không tìm thấy dữ liệu 'EURO FX' sau khi lọc.")

    # 3. CHUẨN HÓA CỘT VÀ ÁNH XẠ

    # Nếu không có header, phải gán tên cột cho các chỉ số vị thế theo Index
    if not is_header_present:
        COL_INDICES_FULL = {
            # 0 đã gán, 2 là Date
            2: 'Report_Date_as_YYYY-MM-DD',
            7: 'Open_Interest_All',
            8: 'Dealer_Positions_Long_All',
            9: 'Dealer_Positions_Short_All',
            14: 'Lev_Money_Positions_Long_All',
            15: 'Lev_Money_Positions_Short_All',
        }
        df_euro.rename(columns=COL_INDICES_FULL, inplace=True)

    # Lấy các cột cần thiết và đổi tên
    df_clean = df_euro[list(COLUMN_MAPPING.keys())].rename(columns=COLUMN_MAPPING)

    # 4. TÍNH TOÁN CHỈ SỐ CƠ BẢN
    df_clean['Date'] = pd.to_datetime(df_clean['Date'])

    # Dealer Net
    df_clean['Dealer_Net'] = df_clean['Dealer_Long_All'] - df_clean['Dealer_Short_All']
    df_clean['Dealer_%OI'] = (df_clean['Dealer_Net'] / df_clean['Open_Interest_All']) * 100

    # CTA Net
    df_clean['CTA_Net'] = df_clean['CTA_Long_All'] - df_clean['CTA_Short_All']
    df_clean['CTA_%OI'] = (df_clean['CTA_Net'] / df_clean['Open_Interest_All']) * 100

    # Giữ lại các cột cần xuất
    FINAL_COLS = ['Date', 'Open_Interest_All', 'Dealer_Net', 'Dealer_%OI', 'CTA_Net', 'CTA_%OI']

    return df_clean[FINAL_COLS]


# =========================================================
# CELL 3: THỰC THI GỘP TẤT CẢ FILE VÀ LƯU
# =========================================================

def combine_and_save_cftc():
    """Chức năng: Quét, xử lý, gộp tất cả file CFTC và lưu ra file filtered."""

    FILE_NAMES = [f for f in os.listdir(RAW_CFTC_PATH) if f.endswith('.txt') or f.endswith('.csv')]
    all_data = []

    print(f"--- BẮT ĐẦU XỬ LÝ {len(FILE_NAMES)} FILE CFTC RAW ---")

    for filename in FILE_NAMES:
        file_path = os.path.join(RAW_CFTC_PATH, filename)

        try:
            df_processed = process_cftc_file(file_path)
            all_data.append(df_processed)
            print(f"✅ Xử lý thành công: {filename}")

        except Exception as e:
            # Nếu có lỗi, chỉ in ra cảnh báo và bỏ qua file lỗi đó
            print(f"⚠️ BỎ QUA file lỗi {filename}: {e}")
            continue

    # Gộp dữ liệu và làm sạch
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        combined_df = combined_df.sort_values('Date').drop_duplicates(subset=['Date'])

        # Lưu ra file cuối cùng
        output_file = os.path.join(FILTERED_PATH, 'cftc_dealer_cta_filtered.csv')
        combined_df.to_csv(output_file, index=False)

        print(f"\n--- HOÀN THÀNH XỬ LÝ CFTC ---")
        print(f"Lưu file tại: {output_file}")
        print(f"Kích thước dữ liệu cuối cùng: {combined_df.shape}")

    else:
        print("\n❌ LỖI NGHIÊM TRỌNG: Không có file nào được xử lý thành công.")

# Chạy hàm chính
combine_and_save_cftc()

--- BẮT ĐẦU XỬ LÝ 5 FILE CFTC RAW ---
✅ Xử lý thành công: FinFut_2022.txt
✅ Xử lý thành công: FinFut_2023.txt
✅ Xử lý thành công: FinFut_2024.txt
✅ Xử lý thành công: FinFut_2025.txt
✅ Xử lý thành công: FinFut_21102025.txt

--- HOÀN THÀNH XỬ LÝ CFTC ---
Lưu file tại: /content/drive/MyDrive/EURUSD_2026/01_Raw/filtered/cftc_dealer_cta_filtered.csv
Kích thước dữ liệu cuối cùng: (199, 6)


In [4]:
# =========================================================
# CELL 3: SETUP & MASTER JOIN (PHIÊN BẢN CUỐI CÙNG ĐÃ SỬA LỖI TẦN SUẤT & ATTRIBUTE)
# =========================================================

import pandas as pd
import numpy as np
import os
from google.colab import drive

# --- A. MOUNT DRIVE & ĐỊNH NGHĨA ĐƯỜNG DẪN ---
try:
    drive.mount('/content/drive')
except ValueError:
    print("Drive đã được Mount. Bỏ qua Mount.")

BASE_PATH = '/content/drive/MyDrive/EURUSD_2026'
RAW_PATH = os.path.join(BASE_PATH, '01_Raw')
FILTERED_PATH = os.path.join(RAW_PATH, 'filtered')
CLEAN_PATH = os.path.join(BASE_PATH, '02_Clean')
os.makedirs(CLEAN_PATH, exist_ok=True)

# --- B. ĐỌC DỮ LIỆU CFTC ĐÃ LỌC (Cột chính) ---
CFTC_FILTERED_FILE = os.path.join(FILTERED_PATH, 'cftc_dealer_cta_filtered.csv')

# Đọc file CFTC, set index là Date (Tần suất tuần)
df_master = pd.read_csv(CFTC_FILTERED_FILE, parse_dates=['Date']).set_index('Date', drop=True)

# Lấy MASK của các ngày báo cáo CFTC gốc để lọc sau khi điền dữ liệu (Bước 2 của phần E)
cftc_original_index = df_master.index.copy()
print(f"✅ Đã đọc dữ liệu CFTC đã lọc. Kích thước ban đầu: {df_master.shape}")


# --- C. ĐỌC VÀ GỘP DGS10 (SỬ DỤNG GỘP NGOÀI) ---
FRED_DIR = os.path.join(RAW_PATH, 'fred')
DGS10_FILE = os.path.join(FRED_DIR, 'FRED_DGS10.csv')
col_name = 'DGS10'

df_fred = pd.read_csv(DGS10_FILE, index_col=0, parse_dates=True, header=0)
df_fred.columns = [col_name]

# Gộp DGS10 hàng ngày vào CFTC (dùng outer để không mất dữ liệu DGS10)
df_master = df_master.join(df_fred, how='outer')
print(f"✅ Đã gộp FRED DGS10 (Gộp Outer).")


# --- D. ĐỌC VÀ GỘP YFINANCE (SỬ DỤNG GỘP NGOÀI) ---
YFINANCE_CLEAN_FILE = os.path.join(FILTERED_PATH, 'yfinance_EURUSD_clean.csv')
df_yf_clean = pd.read_csv(YFINANCE_CLEAN_FILE, index_col=0, parse_dates=True, header=0)
df_euraud_close = df_yf_clean['EURUSD_Close']

# Gộp YFinance hàng ngày vào Master DF
df_master = df_master.join(df_euraud_close, how='outer')
print("✅ Đã gộp tỷ giá EURUSD đã chuẩn hóa (Gộp Outer).")

# --- E. LÀM SẠCH, ĐIỀN DỮ LIỆU, VÀ CHUẨN HÓA MASTER DF ---

df_master.index = pd.to_datetime(df_master.index)
df_master = df_master.sort_index()

# 1. ĐIỀN TẤT CẢ DỮ LIỆU: Điền tiến (ffill) và điền lùi (bfill)
# Đảm bảo các cột DGS10 và EURUSD có giá trị trên tất cả các ngày
df_master = df_master.ffill().bfill()

# 2. LỌC: CHỈ GIỮ LẠI CÁC DÒNG CÓ NGÀY BÁO CÁO CFTC BAN ĐẦU
# Sau khi điền, Master DF có cả ngày CFTC và các ngày giữa hai kỳ báo cáo.
# Ta lọc lại, chỉ giữ các dòng có ngày nằm trong Index CFTC ban đầu.
df_master = df_master[df_master.index.isin(cftc_original_index)]

print(f"\n--- TỔNG QUAN MASTER DATAFRAME ---")
print(f"Kích thước: {df_master.shape}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Đã đọc dữ liệu CFTC đã lọc. Kích thước ban đầu: (199, 5)
✅ Đã gộp FRED DGS10 (Gộp Outer).
✅ Đã gộp tỷ giá EURUSD đã chuẩn hóa (Gộp Outer).

--- TỔNG QUAN MASTER DATAFRAME ---
Kích thước: (199, 7)


In [5]:
# =========================================================
# CELL 4: TÍNH TOÁN CÁC CHỈ SỐ SHEET 1 (ĐÃ BAO GỒM PHASE & TENY CHUẨN)
# =========================================================

# Định nghĩa các cửa sổ (chuẩn hóa theo tuần)
WINDOW_3Y = 156
WINDOW_1Y = 52
WINDOW_1M = 4

# --- A. TÍNH TOÁN CÁC CHỈ SỐ CFTC (SHEET 1) ---

# 1. Dealer %OI và Z3Y_Dealer
df_master['Dealer_Net_OI'] = df_master['Dealer_Net'] / df_master['Open_Interest_All'] * 100
df_master['Dealer_%OI'] = df_master['Dealer_Net_OI']
df_master['Z3Y_Dealer'] = (
    df_master['Dealer_%OI'] - df_master['Dealer_%OI'].rolling(WINDOW_3Y).mean()
) / df_master['Dealer_%OI'].rolling(WINDOW_3Y).std()

# 2. CTA %OI và Z3Y_CTA
df_master['CTA_Net_OI'] = df_master['CTA_Net'] / df_master['Open_Interest_All'] * 100
df_master['CTA_%OI'] = df_master['CTA_Net_OI']
df_master['Z3Y_CTA'] = (
    df_master['CTA_%OI'] - df_master['CTA_%OI'].rolling(WINDOW_3Y).mean()
) / df_master['CTA_%OI'].rolling(WINDOW_3Y).std()

# 3. Dealer ROC (Thay vì Imbalance ROC)
df_master['Dealer_Net_Shift4'] = df_master['Dealer_Net'].shift(WINDOW_1M)
df_master['Dealer_ROC'] = np.where( # Tên cột chuẩn hóa là Dealer_ROC
    df_master['Dealer_Net_Shift4'].abs() != 0,
    (df_master['Dealer_Net'] - df_master['Dealer_Net_Shift4']) / df_master['Dealer_Net_Shift4'].abs() * 100,
    0
)

# 4. CTA Speed
df_master['CTA_Net_Shift4'] = df_master['CTA_Net'].shift(WINDOW_1M)
df_master['CTA_Speed_Denominator'] = df_master['CTA_Net_Shift4'].abs().rolling(WINDOW_1Y).mean()
df_master['CTA_Speed'] = np.where(
    df_master['CTA_Speed_Denominator'] != 0,
    (df_master['CTA_Net'] - df_master['CTA_Net_Shift4']) / df_master['CTA_Speed_Denominator'],
    0
)

# 5. CTA vs Price Divergence
df_master['EURUSD_Pct_Change_4W'] = df_master['EURUSD_Close'].pct_change(WINDOW_1M)
df_master['CTA_Pct_Change_4W'] = df_master['CTA_Net'].pct_change(WINDOW_1M)
df_master['Corr_CTA_Price'] = df_master['CTA_Pct_Change_4W'].rolling(WINDOW_1Y).corr(df_master['EURUSD_Pct_Change_4W'])
# Đổi tên cột chuẩn hóa thành CTA_Divergence
df_master['CTA_Divergence'] = df_master['Corr_CTA_Price'].clip(-5, 5)


# --- B. TÍNH TOÁN CÁC CHỈ SỐ TENY (3 CỘT: Yield, Z3Y, Phase) ---

# 6a. TenY_Yield (Giá trị DGS10 gốc)
df_master['TenY_Yield'] = df_master['DGS10']

# 6b. Z3Y_TenY (Chuẩn hóa)
df_master['DGS10_Mean_156'] = df_master['DGS10'].rolling(WINDOW_3Y).mean()
df_master['DGS10_Std_156'] = df_master['DGS10'].rolling(WINDOW_3Y).std()
df_master['Z3Y_TenY'] = (df_master['DGS10'] - df_master['DGS10_Mean_156']) / df_master['DGS10_Std_156']

# 6c. TenY_Phase (Dựa trên Z3Y_TenY)
CONDITIONS_TENY = [
    (df_master['Z3Y_TenY'] <= -1.5),
    (df_master['Z3Y_TenY'] > -1.5)
]
PHASE_TENY_VALUES = ['Rebuild (risk-on)', 'Unwind (risk-off)']
df_master['TenY_Phase'] = np.select(CONDITIONS_TENY, PHASE_TENY_VALUES, default='Neutral')


# --- C. TÍNH TOÁN CÁC CHỈ SỐ PHASE (2 CỘT: Dealer & CTA) ---

PHASE_VALUES = ['Rebuild', 'Accumulate', 'Crowded', 'Unwind', 'Rebuild']

# 7. Dealer_Phase
CONDITIONS_DEALER = [
    (df_master['Z3Y_Dealer'] <= -1.8),
    (df_master['Z3Y_Dealer'] > -1.8) & (df_master['Z3Y_Dealer'] <= -0.8),
    (df_master['Z3Y_Dealer'] > -0.8) & (df_master['Z3Y_Dealer'] <= 1.5),
    (df_master['Z3Y_Dealer'] > 1.5) & (df_master['Z3Y_Dealer'] <= 1.8),
    (df_master['Z3Y_Dealer'] > 1.8)
]
df_master['Dealer_Phase'] = np.select(CONDITIONS_DEALER, PHASE_VALUES, default='Neutral')

# 8. CTA_Phase
CONDITIONS_CTA = [
    (df_master['Z3Y_CTA'] <= -1.8),
    (df_master['Z3Y_CTA'] > -1.8) & (df_master['Z3Y_CTA'] <= -0.8),
    (df_master['Z3Y_CTA'] > -0.8) & (df_master['Z3Y_CTA'] <= 1.5),
    (df_master['Z3Y_CTA'] > 1.5) & (df_master['Z3Y_CTA'] <= 1.8),
    (df_master['Z3Y_CTA'] > 1.8)
]
df_master['CTA_Phase'] = np.select(CONDITIONS_CTA, PHASE_VALUES, default='Neutral')


print("✅ Đã tính toán đầy đủ 13 cột chỉ số chuẩn cho Sheet 1.")

✅ Đã tính toán đầy đủ 13 cột chỉ số chuẩn cho Sheet 1.


In [6]:
# =========================================================
# CELL 5: LỌC VÀ XUẤT DỮ LIỆU SẠCH (SHEET 1 - 13 CỘT)
# =========================================================

# Các chỉ số cần có của Sheet 1 (Đầy đủ 13 cột theo tài liệu chi tiết)
SHEET1_COLS = [
    'Dealer_%OI',
    'Z3Y_Dealer',
    'Dealer_Phase',
    'Dealer_ROC',
    'CTA_%OI',
    'Z3Y_CTA',
    'CTA_Phase',
    'CTA_Speed',
    'CTA_Divergence',
    'TenY_Yield',
    'Z3Y_TenY',
    'TenY_Phase'
]

# Lọc DataFrame chỉ lấy các cột Sheet 1
df_dealer_cta_clean = df_master[SHEET1_COLS].copy()

# Loại bỏ các hàng có ít nhất một giá trị NaN (Do cửa sổ rolling 156 tuần)
# .reset_index() đưa cột Date từ index trở thành cột đầu tiên tên là 'index'
df_dealer_cta_clean = df_dealer_cta_clean.dropna(how='any').reset_index()

# !!! CHỈNH SỬA TẠI ĐÂY: Đổi tên cột 'index' thành 'Date'
df_dealer_cta_clean.rename(columns={'index': 'Date'}, inplace=True)

# Xuất ra file clean cho Sheet 1
OUTPUT_FILE_DEALER_CTA = os.path.join(CLEAN_PATH, 'dealer_cta_clean.csv')
df_dealer_cta_clean.to_csv(OUTPUT_FILE_DEALER_CTA, index=False)

print(f"\n✅ Xuất file dealer_cta_clean.csv thành công ({df_dealer_cta_clean.shape[0]} dòng).")
print(f"Đã đổi tên cột đầu tiên thành 'Date'.")
print(f"File được lưu tại: {OUTPUT_FILE_DEALER_CTA}")


✅ Xuất file dealer_cta_clean.csv thành công (44 dòng).
Đã đổi tên cột đầu tiên thành 'Date'.
File được lưu tại: /content/drive/MyDrive/EURUSD_2026/02_Clean/dealer_cta_clean.csv
